In [11]:
import re
import os
from textwrap import dedent
from pprint import pprint
from typing import List

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, RunnableLambda, chain
from pydantic import BaseModel, Field

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [12]:
# 1. 벡터 DB 구축
def create_cafe_vector_db():
    loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
    documents = loader.load()

    def split_menu_items(document):
        pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
        items = re.findall(pattern, document.page_content, re.DOTALL)
        return [
            Document(
                page_content=item.strip(),
                metadata={
                    "source": document.metadata.get("source", ""),
                    "menu_number": i,
                    "menu_name": item.split('\n')[0].split('.', 1)[1].strip()
                }
            )
            for i, item in enumerate(items, 1)
        ]

    menu_documents = [doc for d in documents for doc in split_menu_items(d)]
    embeddings = OllamaEmbeddings(model="bge-m3:latest")
    db = FAISS.from_documents(menu_documents, embeddings)
    db.save_local("./db/cafe_db")
    return db

In [13]:
# 2. 도구 정의
@tool
def tavily_search_func(query: str) -> str:
    """인터넷에서 최신 정보나 외부 일반 정보를 검색합니다."""
    tavily = TavilySearchResults(max_results=2)
    docs = tavily.invoke(query)
    if not docs:
        return "관련 정보를 찾을 수 없습니다."

    return "\n---\n".join(
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    )

def wiki_search_and_summarize(input_data: dict):
    loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    docs = loader.load()
    return [
        f'<Document source="{doc.metadata.get("source", "Wikipedia")}"/>\n{doc.page_content}\n</Document>'
        for doc in docs
    ]

class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search and summarization"""
    query: str = Field(..., description="Wikipedia에서 검색할 쿼리입니다.")

summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | ChatPromptTemplate.from_template("Summarize the following text:\n\n{context}\n\nSummary:")
    | ChatOpenAI(
        base_url="https://api.groq.com/openai/v1",
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        temperature=0.7
    )
)

wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        위키백과에서 정보를 찾아 요약합니다.
        배경지식이나 일반적인 정보를 확인할 때 유용합니다.
    """),
    args_schema=WikiSummarySchema
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """카페 메뉴 벡터 DB에서 유사 항목을 검색합니다."""
    embeddings = OllamaEmbeddings(model="bge-m3:latest")
    db = FAISS.load_local("./db/cafe_db", embeddings, allow_dangerous_deserialization=True)
    results = db.similarity_search(query, k=2)
    return results if results else [Document(page_content="관련 카페 메뉴 정보를 찾을 수 없습니다.")]


In [14]:
# 3. LLM 설정 및 도구 바인딩
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
llm_with_tools = llm.bind_tools(tools=tools)

In [15]:
# 4. 도구 호출 체인
@chain
def cafe_search_chain(user_input: str, config: RunnableConfig):
    ai_msg = llm_with_tools.invoke(user_input, config=config)

    tool_results = []
    for call in ai_msg.tool_calls:
        name = call["name"]
        if name == "tavily_search_func":
            tool_results.append(tavily_search_func.invoke(call, config=config))
        elif name == "wiki_summary":
            tool_results.append(wiki_summary.invoke(call, config=config))
        elif name == "db_search_cafe_func":
            tool_results.append(db_search_cafe_func.invoke(call, config=config))

    tool_output = "\n\n".join(str(r.content) for r in tool_results)
    final_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful cafe assistant. Provide accurate information."),
        ("human", "{user_input}"),
        ("ai", ai_msg.content or "도구를 사용해 정보를 찾았습니다."),
        ("human", "검색 결과: {tool_results}")
    ])
    return (final_prompt | llm).invoke({
        "user_input": user_input,
        "tool_results": tool_output
    }, config=config)

In [16]:
# 5. 실행
if __name__ == "__main__":
    try:
        create_cafe_vector_db()
        print("카페 메뉴 벡터 DB 생성 완료.")
    except Exception as e:
        print(f"DB 생성 오류: {e}")

    query = "아메리카노의 가격과 유래가 궁금해요."
    response = cafe_search_chain.invoke(query)
    print("질문:", query)
    print("답변:", response.content)

카페 메뉴 벡터 DB 생성 완료.
질문: 아메리카노의 가격과 유래가 궁금해요.
답변: 아메리카노의 가격과 유래에 대해 알려드리겠습니다.

아메리카노는 에스프레소에 뜨거운 물을 추가하여 희석시킨 커피입니다. 이름의 유래는 2차 세계 대전 당시 이탈리아에 파견된 미국 군인들이 에스프레소가 너무 강하다고 느껴 물을 추가하여 마셨던 것에서 비롯되었다고 합니다.

국내 카페의 아메리카노 가격은 브랜드와 지역에 따라 다르지만, 평균적으로 4,000원에서 6,000원 사이입니다. 스타벅스, 카페베네, 이디야 등 대형 브랜드의 경우 5,000원대, 중소형 카페의 경우 4,000원대에 판매하는 경우가 많습니다.

보다 자세한 정보가 필요하시면 언제든지 질문해 주세요.
